In [3]:
import hopsworks
import glob
import pandas as pd

In [2]:
project = hopsworks.login(
    host="2a5f8040-2d0d-11ed-b5c5-c151c2fe58c1.cloud.hopsworks.ai",
    project="electricity",
    api_key_value="xIGaDiwBZpXxLYRQ.SgheZwqHTZteASnO1PtPMC4HMgco2OVD6VK6xZN3sKCy96FQT27rF0JbBuFivTTA",
)
fs = project.get_feature_store()

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://2a5f8040-2d0d-11ed-b5c5-c151c2fe58c1.cloud.hopsworks.ai:443/p/7287


Connected. Call `.close()` to terminate connection gracefully.


In [67]:
flow_dfs = [pd.read_csv(f) for f in glob.glob("data/Cross*")]
flow_df = flow_dfs[0]
for df in flow_dfs[1:]:
    flow_df = flow_df.merge(df)

In [68]:
flow_df

,Time (CET/CEST),BZN|DK1-NO1 > BZN|SE3 [MW],BZN|SE3 > BZN|DK1-NO1 [MW],BZN|SE4 > BZN|SE3 [MW],BZN|SE3 > BZN|SE4 [MW],BZN|DK1 > BZN|SE3 [MW],BZN|SE3 > BZN|DK1 [MW],BZN|SE3 > BZN|SE2 [MW],BZN|SE2 > BZN|SE3 [MW],BZN|FI > BZN|SE3 [MW],BZN|SE3 > BZN|FI [MW],BZN|NO1 > BZN|SE3 [MW],BZN|SE3 > BZN|NO1 [MW]
0,01.01.2022 00:00 - 01.01.2022 01:00,n/e,n/e,0,1601,0,407,0,4836,0,837,0,1038
1,01.01.2022 01:00 - 01.01.2022 02:00,n/e,n/e,0,1341,0,400,0,4421,0,625,0,1065
2,01.01.2022 02:00 - 01.01.2022 03:00,n/e,n/e,0,1788,0,393,0,4618,0,495,0,927
3,01.01.2022 03:00 - 01.01.2022 04:00,n/e,n/e,0,2349,0,293,0,4749,0,344,0,748
4,01.01.2022 04:00 - 01.01.2022 05:00,n/e,n/e,0,1798,0,302,0,4229,0,200,0,918
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8818,31.12.2022 19:00 - 31.12.2022 20:00,n/e,n/e,-,-,-,-,-,-,-,-,-,-
8819,31.12.2022 20:00 - 31.12.2022 21:00,n/e,n/e,-,-,-,-,-,-,-,-,-,-
8820,31.12.2022 21:00 - 31.12.2022 22:00,n/e,n/e,-,-,-,-,-,-,-,-,-,-
8821,31.12.2022 22:00 - 31.12.2022 23:00,n/e,n/e,-,-,-,-,-,-,-,-,-,-


In [69]:
flow_df.rename(columns={'Time (CET/CEST)': 'time'}, inplace=True)
flow_df['time'] = pd.to_datetime(flow_df['time'].apply(lambda x: x.split("-")[0].strip()), format='%d.%m.%Y %H:%M')

In [70]:
flow_df.rename(columns={c: c.replace("BZN|", "").replace("[MW]", "").replace(" > ", "").replace("-", "").replace(" ", "") for c in flow_df.columns}, inplace=True)

In [71]:
flow_columns = [c for c in flow_df.columns if c not in ['time']]

In [72]:
for column in flow_columns:
    # Cast to float
    flow_df[column] = pd.to_numeric(flow_df[column], errors="coerce").fillna(0)
    
    # Invert outgoing flows
    if not column.startswith("SE3"):
        flow_df[column] = flow_df[column] * -1

In [73]:
flow_df['zone'] = "SE3"

In [74]:
se_flow_fg  = fs.get_or_create_feature_group(
    name="se_cross_region_flows",
    version=1,
    description="Sweden cross region flows",
    primary_key=["zone"],
    event_time="time",
    online_enabled=False,
    statistics_config={
        'histograms': True, 
        'correlations': True
    }
)

In [75]:
se_flow_fg.insert(flow_df)

Feature Group created successfully, explore it at 
https://2a5f8040-2d0d-11ed-b5c5-c151c2fe58c1.cloud.hopsworks.ai:443/p/7287/fs/7235/fg/14356


Uploading Dataframe: 0.00% |          | Rows 0/8823 | Elapsed Time: 00:00 | Remaining Time: ?

Launching offline feature group backfill job...
Backfill Job started successfully, you can follow the progress at 
https://2a5f8040-2d0d-11ed-b5c5-c151c2fe58c1.cloud.hopsworks.ai/p/7287/jobs/named/se_cross_region_flows_1_offline_fg_backfill/executions


(<hsfs.core.job.Job at 0x7fe894bd2970>, None)